## Testes da rotina da classe das Equações de Mackey-Glass

## 1. Importando as bibliotecas necessárias

### Bibliotecas obrigatórias

In [1]:
import numpy as np
from jitcdde import jitcdde, y, t

%matplotlib widget
import matplotlib.pyplot as plt

import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 100

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

plt.style.use('dark_background')

### Bibliotecas não-obrigatórias

In [2]:
import sys 
sys.path.insert(0, '../../scripts')
import mackeyglassequations as mgeq

## 2. Definindo a função (debug)

Não é necessário executar essa linha se você estiver importando o arquivo com a classe.

In [296]:
class MackeyGlass:
    
    def __init__(self, p_iniciais, tau, gamma=0.1, beta=0.2, theta=1, n=10, dt=0.0001):
        """
        Descrição:
        ----------
        Construtor da classe 'SistemaLorenz'

        Parâmetros:
        -----------
        p_iniciais: np.ndarray
            Valores iniciais da concentração de glóbulos vermelhos no sangue, o numero de valores deve ser igual à Tau
        tau: int
            Parâmetro das Equações de Mackey-Glass
        gamma: int ou float
            Parâmetro das Equações de Mackey-Glass
        beta: int ou float
            Parâmetro das Equações de Mackey-Glass
        theta: int ou float
            Parâmetro das Equações de Mackey-Glass
        n: int
            Parâmetro das Equações de Mackey-Glass
        dt: float
            Tamanho do diferencial de tempo que iremos utilizar nos cálculos, ou seja, a resolução temporal de nossa solução
            
        Retorna:
        --------
        Nada
        """
        
        if not (type(p_inicial) is np.ndarray):
            raise TypeError("0 valor da concentração de glóbulos vermelhos no sangue deve ser um array do numpy!")
        
        if not (((type(gamma) is int) | (type(gamma) is float)) & (gamma > 0)):
            raise TypeError("Gamma deve ser um int ou float positivo!")

        if not (((type(beta) is int) | (type(beta) is float)) & (beta > 0)):
            raise TypeError("Beta deve ser um int ou float positivo!")

        if not (((type(theta) is int) | (type(theta) is float)) & (theta > 0)):
            raise TypeError("Theta deve ser um int ou float positivo!")
                
        if not (((type(tau) is int) | (type(tau) is float)) & (tau > 0)):
            raise TypeError("Tau deve ser um int ou float positivo!")
                
        if not ((type(n) is int) & (n > 0)):
            raise TypeError("n deve ser um int positivo!")
            
        if not ((type(dt) is float) & (dt > 0)):
            raise TypeError("dt deve ser um float positivo!")
        
        self._p_inicial = p_inicial
        self._gamma = gamma
        self._beta = beta
        self._theta = theta
        self._tau = tau
        self._n = n
        self._dt = dt
        pass
    
    def _equacao(self):
        """
        Descrição:
        ----------
        Função interna que retorna as equações de Mackey-Glass calculadas no instante t atual
        
        Parâmetros:
        -----------
        estado_atual: np.ndarray
            Vetor das posições (p_atual, p_delay) atuais do sistema    
        t: float
            Instante t no qual estamos calculando as derivadas
            
        Retorna:
        --------
        As equações de Mackey-Glass para os parâmetros estimados na forma de um array
        """     
        
        gamma = self._gamma
        beta = self._beta
        theta = self._theta
        tau = self._tau
        n = self._n
        
        dp_dt = (beta*((theta)**n)*y(0,t-tau))/(((theta)**n) + (y(0,t-tau)**n)) - gamma*y(0)
        return [dp_dt]
    
    def calcular(self, t_inicial, t_final):
        """
        Descrição:
        ----------
        Evolui as equações de Mackey-Glass com base nas condições iniciais configuradas, para um t indo de t_inicial até t_final, com n_instantes calculados
        
        Parâmetros:
        -----------
        t_inicial: int
            Dia em que iniciamos os cálculos
        t_final: int
            Dia em que terminamos os cálculos
            
        Retorna:
        --------
        Um vetor com as soluções estimadas e um vetor com os instantes temporais utilizados
        """
        
        if not ((type(t_inicial) is int) & (t_inicial >= 0)):
            raise TypeError("t_inicial deve ser um int não nulo!")
            
        if not ((type(t_final) is int) & (t_final > 0)):
            raise TypeError("t_final deve ser um int positivo!")
        
        p_anteriores = self._p_delays
        dt = self._dt
        
        p_iniciais = np.append(p_anteriores, self._p_inicial)
        p_anteriores_derivada = np.zeros(len(p_iniciais))
        t_anteriores = np.arange(-len(p_iniciais)+1, 1, 1)
        condicoes_iniciais = []
        
        for i in range(0, len(p_iniciais), 1):
            condicao = (t_anteriores[i], p_iniciais[i], p_anteriores_derivada[i])
            condicoes_iniciais.append(condicao)
        
        equacao = self._equacao()
        DDE = jitcdde(equacao)
        
        DDE.add_past_points(condicoes_iniciais)
        DDE.step_on_discontinuities()
        
        n_instantes = int((t_final - t_inicial)/dt)        
        instantes_temporais = np.linspace(t_inicial, t_final, n_instantes)
        t_integracao = DDE.t + instantes_temporais
        
        solucoes = []
        for t in t_integracao:
            solucoes.append(DDE.integrate(t))
        
        return solucoes, instantes_temporais, p_anteriores

## 3. Testando a classe

In [290]:
t_inicial = 0
t_final = 800
dt = 0.001
tau = 30

In [291]:
p_inicial = 0.8

In [292]:
macglass = MackeyGlass(p_inicial, tau=tau, dt=dt)

In [293]:
solucoes, instantes_temporais, p_iniciais = macglass.calcular(t_inicial = t_inicial, t_final = t_final)

Generating, compiling, and loading C code.
Using default integration parameters.


/mnt/C492FBF292FBE6B8/Unicamp/pibic/pibic-2020-2021/pibic2020-env/lib/python3.8/site-packages/jitcdde/_jitcdde.py:791: UserWarning: The target time is smaller than the current time. No integration step will happen. The returned state will be extrapolated from the interpolating Hermite polynomial for the last integration step. You may see this because you try to integrate backwards in time, in which case you did something wrong. You may see this just because your sampling step is small, in which case there is no need to worry.
  warn("The target time is smaller than the current time. No integration step will happen. The returned state will be extrapolated from the interpolating Hermite polynomial for the last integration step. You may see this because you try to integrate backwards in time, in which case you did something wrong. You may see this just because your sampling step is small, in which case there is no need to worry.")


### 3.a) Série Temporal de Mackey-Glass

In [294]:
fig, ax = plt.subplots()
fig.suptitle("Série temporal de 0 a 800 dias da equação de Mackey-Glass para\n ")
ax.plot(instantes_temporais, solucoes, color='DeepSkyBlue')

ax.set_ylabel('P(t)')
ax.set_xlabel('t')
ax.set_xlim(0,)
    
ax.grid(True)

fig.tight_layout()
sns.despine()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 3.b) Atrator de Mackey-Glass